# AiiDA-KKR demo

Here is a Demo to run the Voronoi code with a follow up KKR calculation with AiiDA 
with pure python code

Some Comments:

In oder to run the KKR and Voronoi codes you have to set them up as codes in AiiDA.
You might source a bash rc in the pretext execution of the code for Licenzing issues.
Also you should symbol link the ElementPotential DataBase for the Voronoi code
    
If you want to test, use submit test, which will save all files created before a calculation would be run in a local test_submit folder

In [39]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import time
import os
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.orm import Code, load_node
from aiida.orm import DataFactory, CalculationFactory
from aiida_kkr.tools.kkrcontrol import write_kkr_inputcard_template, fill_keywords_to_inputcard, create_keyword_default_values
from pprint import pprint
from scipy import array
from aiida_kkr.calculations.kkr import KkrCalculation
from aiida_kkr.calculations.voro import VoronoiCalculation
from aiida_kkr.parsers.voro import VoronoiParser
from aiida_kkr.parsers.kkr import KkrParser

ParameterData = DataFactory('parameter')
StructureData = DataFactory('structure')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# Prepare and AiiDAStructure Data as input, example Cu

alat = 6.830000 # in a_Bohr
abohr = 0.52917721067
# number of atom positions in unit cell
natyp = 1
# bravais vectors
bravais = array([[0.5, 0.5, 0.0], [0.5, 0.0, 0.5], [0.0, 0.5, 0.5]])

a = 0.5*alat*abohr
Cu = StructureData(cell=[[a, a, 0.0], [a, 0.0, a], [0.0, a, a]])
Cu.append_atom(position=[0.0, 0.0, 0.0], symbols='Cu')
#Cu.store()
Cu = load_node(79546)
print(Cu)

0.5


In [5]:
# Now gernerate a ParameterData node with keyvalues needed by voronoi and KKR 
# we use a helper function for some defaults and set some values

keywords = create_keyword_default_values()
keywords['NATYP'][0] = natyp
keywords['ALATBASIS'][0] = alat
keywords['NSPIN'][0] = 1
keywords['LMAX'][0] = 2
# choose only coarse energy contour and k-mesh for test purposes
keywords['NPOL'][0] = 4
keywords['NPT1'][0] = 3
keywords['NPT2'][0] = 10
keywords['NPT3'][0] = 3
keywords['BZKX'][0] = 10
keywords['BZKY'][0] = 10
keywords['RCLUSTZ'][0] = 1.50
keywords['RCLUSTXY'][0] = 1.50
# for ASA
keywords['INS'] = [0, '%i']
keywords['KSHAPE'] = [0, '%i']
pprint(keywords)

{'<NLBASIS>': [1, '%i'],
 '<NRBASIS>': [1, '%i'],
 '<RBLEFT>': [[0, 0, 0], ['%f', '%f', '%f']],
 '<RBRIGHT>': [[0, 0, 0], ['%f', '%f', '%f']],
 'ALATBASIS': [6.83, '%f'],
 'BZKX': [10, '%i'],
 'BZKY': [10, '%i'],
 'BZKZ': [20, '%i'],
 'CARTESIAN': ['F', '%1s'],
 'EMAX': [1.0, '%6.3f'],
 'EMIN': [-0.4, '%6.3f'],
 'GMAX': [65, '%i'],
 'HFIELD': [0.0, '%5.2f'],
 'IMIX': [0, '%i'],
 'INS': [0, '%i'],
 'INTERFACE': ['F', '%1s'],
 'KSHAPE': [0, '%i'],
 'LINIPOL': ['F', '%1s'],
 'LMAX': [2, '%i'],
 'NATYP': [1, '%i'],
 'NCHEB': [13, '%i'],
 'NPAN_EQ': [5, '%i'],
 'NPAN_LOG': [15, '%i'],
 'NPOL': [4, '%4i'],
 'NPT1': [3, '%4i'],
 'NPT2': [10, '%4i'],
 'NPT3': [3, '%4i'],
 'NSPIN': [1, '%i'],
 'NSTEPS': [30, '%i'],
 'QBOUND': ['1D-3', '%s'],
 'RCLUSTXY': [1.5, '%7.2f'],
 'RCLUSTZ': [1.5, '%7.2f'],
 'RMAX': [7, '%i'],
 'R_LOG': [1.0, '%5.1f'],
 'STRMIX': [0.01, '%f'],
 'TEMPR': [800.0, '%7.1f'],
 'ZPERIODL': [[0, 0, 0], ['%f', '%f', '%f']],
 'ZPERIODR': [[0, 0, 0], ['%f', '%f', '%f']]}


In [6]:
# Store the node
keyw = ParameterData(dict=keywords)
#keyw.store()
keyw = load_node(79550)
print keyw

uuid: a5da7eef-32ee-47e2-9d94-821dc231869b (pk: 79550)


In [12]:
# Running a single Vornoi calculation
code = Code.get_from_string('voro@local_mac')
calc = VoronoiCalculation()
calc.label = 'Test voronoi'
calc.set_withmpi(False)
calc.set_resources({"num_machines" : 1})
calc.set_max_wallclock_seconds(300)
calc.set_computer('local_mac')
calc.use_code(code)
calc.use_structure(Cu)
calc.use_parameters(keyw)

In [14]:
submit_test = False

if submit_test:
    subfolder, script_filename = calc.submit_test()
    print "Test_submit for calculation (uuid='{}')".format(
        calc.uuid)
    print "Submit file in {}".format(os.path.join(
        os.path.relpath(subfolder.abspath),
        script_filename
    ))
else:
    calc.store_all()
    print "created calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)
    calc.submit()
    print "submitted calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)

/Users/broeder/aiida/github/submit_test/20171110-00023/inputcard
WARNING for replacement of keyword KSHAPE: not in template!
... also naked keyword not found. Just appending it.
WARNING for replacement of keyword INS: not in template!
... also naked keyword not found. Just appending it.
Test_submit for calculation (uuid='9267e048-2988-41cf-bee2-020b7c546eef')
Submit file in submit_test/20171110-00023/_aiidasubmit.sh


In [15]:
!cat submit_test/20171110-00007/_aiidasubmit.sh
!ls submit_test/20171110-00001/
! cat submit_test/20171110-00007/inputcard

#!/bin/bash



source ~/.profile_aiida
ln -s /Users/broeder/aiida/codes/kkr/voronoi/ElementDataBase

'/Users/broeder/aiida/codes/kkr/voronoi/voronoi.exe'  > 'out_voronoi' 
_aiidasubmit.sh inputcard


In [ ]:
# Ontop the voronoi calculation we want to run a KKR calculation
# for this we have to get some things from the voronoi calculation

In [22]:
# use the calculation run before or load a voronoi calculation
calc2 = load_node(79565)

In [23]:
# We create a new parameter node in which we store the emin extracted form the voronoi calculation
#emin = calc1.get_outputs_dict()['output_parameters'].get_dict()['EMIN']
emin = calc2.res.EMIN
remote = calc2.get_outputs_dict()['remote_folder']
keywords2 = keywords
keywords2['EMIN'][0] = emin


In [24]:
keyw2 = ParameterData(dict=keywords2)
#keyw2.store()
keyw2 = load_node(79570)
print keyw2

uuid: 6b1ef75a-a2c7-4967-9ce9-ed44e0948c8b (pk: 79570)


In [34]:
# Now we create and run the kkr Calculation
code = Code.get_from_string('kkr1@local_mac')#'kkrimp@local_mac')
calc1 = KkrCalculation()
calc1.label = 'Test kkr'
calc1.set_withmpi(False)
calc1.set_resources({"num_machines" : 1})
calc1.set_max_wallclock_seconds(300)
calc1.set_computer('local_mac')
calc1.use_code(code)
#calc1.use_structure(Cu)
calc1.use_parameters(keyw2)
calc1.use_parent_folder(remote)

In [37]:
submit_test = False
if submit_test:
    subfolder, script_filename = calc1.submit_test()
    print "Test_submit for calculation (uuid='{}')".format(
        calc1.uuid)
    print "Submit file in {}".format(os.path.join(
        os.path.relpath(subfolder.abspath),
        script_filename
    ))
else:
    calc1.store_all()
    print "created calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc1.uuid, calc.dbnode.pk)
    calc1.submit()
    print "submitted calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)

created calculation; calc=Calculation(uuid='0f14ff87-163b-4dec-9ada-c9c184b51459') # ID=None
submitted calculation; calc=Calculation(uuid='9267e048-2988-41cf-bee2-020b7c546eef') # ID=None


In [29]:
! cat submit_test/20171110-00020///_aiidasubmit.sh
! ls submit_test/20171110-00020//
! cat submit_test/20171110-00020/inputcard

#!/bin/bash



'/Users/broeder/aiida/codes/kkr/kkr/kkr.x'  > 'out_kkr' 
_aiidasubmit.sh inputcard       potential       shapefun
RUNOPT
<RUNOP1><RUNOP2><RUNOP3><RUNOP4><RUNOP5><RUNOP6><RUNOP7><RUNOP8>
+-------+-------+-------+-------+-------+-------+-------+-------
TESTOPT
<TSTOP1><TSTOP2><TSTOP3><TSTOP4><TSTOP5><TSTOP6><TSTOP7><TSTOP8>
<TSTOP9><TSTP10><TSTP11><TSTP12><TSTP13><TSTP14><TSTP15><TSTP16>
+-------+-------+-------+-------+-------+-------+-------+-------
NSPIN= 1      
LMAX=  2     
BZDIVIDE= 10      10      20    
RCLUSTZ=     1.50  
RCLUSTXY=    1.50   
EMIN= -0.600
EMAX=  1.000
TEMPR=   800.0
NPOL=    4  
NPT1=    3  
NPT2=   10  
NPT3=    3  
NSTEPS= 30      
IMIX= 0     
STRMIX= 0.010000
QBOUND= 1D-3    
LINIPOL= F        
HFIELD=  0.00   
  ALATBASIS= 3.415000    lattice constants a (in a.u.)
--------------------------------------------------------------------------------
   BRAVAIS                   (units of lattice constant)
        1.000000000000000     1.0000000000

In [ ]:
# Check with the verdi shell if everything with you calculations went right

In [ ]:
# Voronoi parser test

In [ ]:
n = load_node(79559)
retrieved_dict = {'retrieved' : n}
voro_parser = VoronoiParser(calc)
suc, nodes = voro_parser.parse_with_retrieved(retrieved_dict)
print suc
print nodes
print nodes[0][1].get_dict()['EMIN']